In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 23.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=ee4600427dd833596284be67d946148895f4f70015f19e804fe993fd4e20e475
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark


In [5]:
from pyspark.sql import SparkSession

In [6]:
spark=SparkSession.builder.master('local[4]').appName('project').getOrCreate()

In [7]:
# Load data
df=spark.read.csv('/content/drive/MyDrive/Project_knowit/finalclassification.csv',header=True,inferSchema=True)

In [8]:
df

DataFrame[_c0: int, year: int, month: int, airport: string, arr_del15: double, carrier_ct: double, weather_ct: double, nas_ct: double, security_ct: double, late_aircraft_ct: double, arr_diverted: double, carrier_delay: double, weather_delay: double, nas_delay: double, security_delay: double, late_aircraft_delay: double, arr_delay: int]

In [9]:
df.count()

78439

In [10]:
df.show(5)

+---+----+-----+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+
|_c0|year|month|airport|           arr_del15|          carrier_ct|          weather_ct|              nas_ct|         security_ct|    late_aircraft_ct|        arr_diverted|       carrier_delay|       weather_delay|           nas_delay|      security_delay| late_aircraft_delay|arr_delay|
+---+----+-----+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+
|  0|2022|   11|    ABY| -0.3317854945166775|-0.37592621851215446|-0.17641855560199077| -0.2618669900411791|-0.20422931220321408|-0.3020701

In [11]:
df.summary().show()

+-------+------------------+-----------------+------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------------+
|summary|               _c0|             year|             month|airport|           arr_del15|          carrier_ct|          weather_ct|              nas_ct|         security_ct|    late_aircraft_ct|        arr_diverted|       carrier_delay|      weather_delay|           nas_delay|      security_delay| late_aircraft_delay|          arr_delay|
+-------+------------------+-----------------+------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-------

In [12]:
pdf = df.toPandas()

In [13]:
pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78439 entries, 0 to 78438
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   _c0                  78439 non-null  int32  
 1   year                 78439 non-null  int32  
 2   month                78439 non-null  int32  
 3   airport              78439 non-null  object 
 4   arr_del15            78439 non-null  float64
 5   carrier_ct           78439 non-null  float64
 6   weather_ct           78439 non-null  float64
 7   nas_ct               78439 non-null  float64
 8   security_ct          78439 non-null  float64
 9   late_aircraft_ct     78439 non-null  float64
 10  arr_diverted         78439 non-null  float64
 11  carrier_delay        78439 non-null  float64
 12  weather_delay        78439 non-null  float64
 13  nas_delay            78439 non-null  float64
 14  security_delay       78439 non-null  float64
 15  late_aircraft_delay  78439 non-null 

In [14]:
df.groupby('arr_delay').count().show()

+---------+-----+
|arr_delay|count|
+---------+-----+
|        1|73934|
|        0| 4505|
+---------+-----+



In [16]:
pdf['arr_delay'].value_counts()

1    73934
0     4505
Name: arr_delay, dtype: int64

In [17]:
pdf.isnull().sum()

_c0                    0
year                   0
month                  0
airport                0
arr_del15              0
carrier_ct             0
weather_ct             0
nas_ct                 0
security_ct            0
late_aircraft_ct       0
arr_diverted           0
carrier_delay          0
weather_delay          0
nas_delay              0
security_delay         0
late_aircraft_delay    0
arr_delay              0
dtype: int64

In [18]:
from pyspark.sql.functions import isnull,when,count,col

In [19]:

df.select([count(when(isnull(c),c)).alias(c) for c in df.columns]).show()

+---+----+-----+-------+---------+----------+----------+------+-----------+----------------+------------+-------------+-------------+---------+--------------+-------------------+---------+
|_c0|year|month|airport|arr_del15|carrier_ct|weather_ct|nas_ct|security_ct|late_aircraft_ct|arr_diverted|carrier_delay|weather_delay|nas_delay|security_delay|late_aircraft_delay|arr_delay|
+---+----+-----+-------+---------+----------+----------+------+-----------+----------------+------------+-------------+-------------+---------+--------------+-------------------+---------+
|  0|   0|    0|      0|        0|         0|         0|     0|          0|               0|           0|            0|            0|        0|             0|                  0|        0|
+---+----+-----+-------+---------+----------+----------+------+-----------+----------------+------------+-------------+-------------+---------+--------------+-------------------+---------+



In [20]:
# Prepare features and label
required_features =['year','month','arr_del15','carrier_ct','weather_ct','nas_ct','security_ct','late_aircraft_ct','arr_diverted','carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay']

In [21]:
pdf[required_features]

,year,month,arr_del15,carrier_ct,weather_ct,nas_ct,security_ct,late_aircraft_ct,arr_diverted,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2022,11,-0.331785,-0.375926,-0.176419,-0.261867,-0.204229,-0.302070,-0.217510,-0.319421,-0.218920,-0.232233,-0.163283,-0.291977
1,2022,11,-0.311897,-0.338844,-0.282602,-0.248319,-0.204229,-0.279732,-0.217510,-0.275219,-0.250517,-0.217853,-0.163283,-0.282159
2,2022,11,-0.338415,-0.375717,-0.282602,-0.286970,-0.204229,-0.284927,-0.217510,-0.319172,-0.250517,-0.242053,-0.163283,-0.265262
3,2022,11,-0.245603,-0.268870,-0.282602,-0.233377,-0.204229,-0.177390,-0.217510,-0.236480,-0.250517,-0.220308,-0.163283,-0.092414
4,2022,11,0.410709,0.224722,0.041687,0.455564,-0.204229,0.491035,0.090656,1.034204,-0.094964,0.133576,-0.163283,0.672502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78434,2019,1,0.105756,-0.122007,-0.014274,0.071048,-0.204229,0.322024,-0.217510,-0.169680,-0.012326,0.031514,-0.163283,0.292327
78435,2019,1,-0.252233,-0.318522,-0.183593,-0.237959,-0.204229,-0.161632,0.090656,-0.239211,-0.212844,-0.202772,-0.163283,-0.232611
78436,2019,1,-0.172680,-0.278088,0.041687,-0.051678,-0.204229,-0.176351,-0.217510,-0.241695,-0.130207,-0.072301,-0.163283,-0.209092
78437,2019,1,-0.113015,-0.232835,0.710355,-0.057854,-0.204229,-0.133232,-0.217510,-0.201714,0.518742,-0.058623,-0.163283,-0.137396


In [22]:
# Import PySpark libraries
from pyspark.ml.feature import VectorAssembler

In [23]:

assembler= VectorAssembler(inputCols=required_features,
                           outputCol='features')

In [24]:
transformed_data=assembler.transform(df)

In [25]:
transformed_data.show(5)

+---+----+-----+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+
|_c0|year|month|airport|           arr_del15|          carrier_ct|          weather_ct|              nas_ct|         security_ct|    late_aircraft_ct|        arr_diverted|       carrier_delay|       weather_delay|           nas_delay|      security_delay| late_aircraft_delay|arr_delay|            features|
+---+----+-----+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+
|  0|2022|   11|    ABY| -0.3317854945166775|-0.37592621851215446|-0.1764185

In [26]:
# Split data into train and test sets
training_data,test_data =transformed_data.randomSplit([0.75,0.25],
                                                      seed=123)

In [27]:
training_data.groupby('arr_delay').count().show()

+---------+-----+
|arr_delay|count|
+---------+-----+
|        1|55622|
|        0| 3381|
+---------+-----+



In [28]:
from pyspark.ml.classification import LogisticRegression


In [29]:
# Define logistic regression model
dt=LogisticRegression(labelCol='arr_delay',featuresCol='features')

In [30]:
# Fit model to training data
model =dt.fit(training_data)

In [82]:
# Make predictions on test data
predictions_test =model.transform(test_data)
predictions_train =model.transform(training_data)

In [83]:
predictions_test.show()

+---+----+-----+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+----------+
|_c0|year|month|airport|           arr_del15|          carrier_ct|          weather_ct|              nas_ct|         security_ct|    late_aircraft_ct|        arr_diverted|       carrier_delay|       weather_delay|           nas_delay|      security_delay| late_aircraft_delay|arr_delay|            features|       rawPrediction|         probability|prediction|
+---+----+-----+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--

In [84]:
predictions_train.show()

+---+----+-----+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+----------+
|_c0|year|month|airport|           arr_del15|          carrier_ct|          weather_ct|              nas_ct|         security_ct|    late_aircraft_ct|        arr_diverted|       carrier_delay|       weather_delay|           nas_delay|      security_delay| late_aircraft_delay|arr_delay|            features|       rawPrediction|         probability|prediction|
+---+----+-----+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--

In [87]:
predictions_test.select('prediction').toPandas()

,prediction
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
19431,1.0
19432,1.0
19433,1.0
19434,1.0


In [91]:
pred_train=predictions_train.select('prediction').toPandas()
pred_test=predictions_test.select('prediction').toPandas()

In [92]:
actual_train= training_data.select('arr_delay').toPandas()
actual_test= test_data.select('arr_delay').toPandas()

In [93]:
from sklearn.metrics import accuracy_score



In [96]:
# Evaluate model performance
Accuracy_train=accuracy_score(actual_train, pred_train)
Accuracy_test=accuracy_score(actual_test, pred_test)


In [97]:
print("Accuracy for test:", Accuracy_test)
print("----------------------------------")
print("Accuracy for train:", Accuracy_train)


Accuracy for test: 1.0
----------------------------------
Accuracy for train: 1.0
